# École Polytechnique de Montréal
# Département Génie Informatique et Génie Logiciel

# INF8460 – Traitement automatique de la langue naturelle - TP3

# Objectifs d’apprentissage
 • Utiliser des plongements lexicaux pré-entrainés pour de la classification
 
 • Entrainer des plongements lexicaux de type word2vec
 
 • Implanter des modèles de classification neuronaux

## Équipe et contributions 
Veuillez indiquer la contribution effective de chaque membre de l'équipe en pourcentage et en indiquant les modules ou questions sur lesquelles chaque membre a travaillé

Cedric Sadeu (1869737): 1/3

Mamoudou Sacko (1924187): 1/3

Oumayma Messoussi (2016797): 1/3

# Librairies externes

In [ ]:
import gensim
from gensim.models import Word2Vec
import io
import nltk
import numpy as np
import os
import pandas as pd
import requests
import sklearn
import sklearn.naive_bayes
import tensorflow as tf
import time
from typing import Dict
import zipfile

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

# Téléchargement et lecture des données

In [ ]:
DATA_PATH = os.path.join(os.getcwd(), "aclImdb")

## Téléchargement

In [ ]:
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xzf aclImdb_v1.tar.gz
!rm aclImdb_v1.tar.gz
!echo Done!

--2020-10-09 14:05:25--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  17.6MB/s    in 7.7s    

2020-10-09 14:05:33 (10.5 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]

Done!


In [ ]:
def download_wikipedia_embeddings() -> None:
    if not os.path.exists(os.path.join(os.getcwd(), "model.txt")):
        res = requests.get("http://vectors.nlpl.eu/repository/11/3.zip")
        with zipfile.ZipFile(io.BytesIO(res.content)) as z:
            z.extractall("./")
        os.remove(os.path.join(os.getcwd(), "3.zip"))
        os.remove(os.path.join(os.getcwd(), "meta.json"))
        os.remove(os.path.join(os.getcwd(), "model.bin"))
        os.remove(os.path.join(os.getcwd(), "README"))

## Lecture

In [ ]:
def read_data(path):
    traintest = ['train', 'test']
    classes = ['pos', 'neg']
    corpus = {cls: [] for cls in classes}

    # Each data is a list of strings(reviews)
    reviews = []
    labels = []
    for cls in classes:
        dir_path = os.path.join(path, cls)
        
        for filename in os.listdir(dir_path):
            file = os.path.join(dir_path, filename)
            with open(file, encoding = 'utf-8') as f:
                corpus[cls].append(f.read().replace("\n", " "))
        
    return corpus

In [ ]:
train_data = read_data(os.path.join(DATA_PATH, 'train'))
test_data = read_data(os.path.join(DATA_PATH, 'test'))

In [ ]:
train_data['pos'][0]

'This is an interesting true story of Archie Grey Owl, Who dreamed of being an Indiain when he was a child until the age of 17 he was born in England then moved to Canada where he was adotped by Indiains and he writes collums in magazines and he wrote a book that caugt the attention of millions the book was of his life. But at the end he told his wife that he was not a real Indiain and she was fine with it and he died at the age of 43 two years after he went back into the wildness.'

In [ ]:
def create_wikipedia_embeddings(word_indices: Dict[str, int], vocab_len: int) -> np.ndarray:
    with open("./model.txt", "r", encoding="UTF-8") as f:
        shape_string = f.readline()
        lines = f.readlines() 
        
    embedding = np.zeros((vocab_len, 300), dtype=float)
    for line in lines:
        splitted_line = line.split(" ")
        word = splitted_line[0].split("_")[0]
        if word in word_indices and word_indices[word] < vocab_len:
            embedding_line = splitted_line[1:]
            embedding[word_indices[word]] = list(map(float, embedding_line))
        
    return embedding

## Prétraitement

In [ ]:
class Preprocess(object):
    def __init__(self, lemmatize=True):
        self.stopwords = set(nltk.corpus.stopwords.words("english"))
        self.lemmatize = lemmatize

    def preprocess_pipeline(self, data):
        clean_tokenized_data = self._clean_doc(data)
        if self.lemmatize:
            clean_tokenized_data = self._lemmatize(clean_tokenized_data)

        return clean_tokenized_data

    def _clean_doc(self, data):
        tokenizer = nltk.tokenize.RegexpTokenizer(r"\w+")
        return [
            [
                token.lower()
                for token in tokenizer.tokenize(review)
                if token.lower() not in self.stopwords
                and len(token) > 1
                and token.isalpha()
                and token != "br]"
            ]
            for review in data
        ]

    def _lemmatize(self, data):
        lemmatizer = nltk.stem.WordNetLemmatizer()
        return [[lemmatizer.lemmatize(word) for word in review] for review in data]

    def convert_to_reviews(self, tokenized_reviews):
        reviews = []
        for tokens in tokenized_reviews:
            reviews.append(" ".join(tokens))

        return reviews

In [ ]:
pre = Preprocess()

train_pos = pre.preprocess_pipeline(train_data["pos"])
train_neg = pre.preprocess_pipeline(train_data["neg"])
test_pos = pre.preprocess_pipeline(test_data["pos"])
test_neg = pre.preprocess_pipeline(test_data["neg"])

y_train = [1] * len(train_pos) + [0] * len(train_neg)
y_test = [1] * len(test_pos) + [0] * len(test_neg)
X_train = [" ".join(sentence) for sentence in train_pos + train_neg]
X_test = [" ".join(sentence) for sentence in test_pos + test_neg]

print("{} training sentences: {} pos and {} neg".format(len(X_train), len(train_pos), len(train_neg)))
print("{} test sentences: {} pos and {} neg".format(len(X_test), len(test_pos), len(test_neg)))

25000 training sentences: 12500 pos and 12500 neg
25000 test sentences: 12500 pos and 12500 neg


# 1. Entrainement de plongements lexicaux

Vous devez réaliser les étapes suivantes:

## a) Utiliser Gensim pour entrainer un modèle word2vec sur le corpus. 

In [ ]:
model = Word2Vec(min_count=5, window=5, size=256, sample=1e-2,
alpha=1e-2, min_alpha=1e-4, negative=5, workers=4)

model.build_vocab(X_train)

start = time.time()
model.train(X_train, total_examples=model.corpus_count, epochs=10)
print(time.time() - start)

179.6863615512848


## b) Décrire les paramètres du ou des modèles entraînés, leur taille sur disque, le nombre de mots encodés, le temps d'entraînement, etc.

## c) Décrire le cas échéant et de manière précise tout problème que vous avez eu à obtenir votre modèle et les façons de résoudre ces problèmes.

## d) Retrouvez les 5 mots voisins des mots suivants : excellent, terrible

In [ ]:
print(model.most_similar('excellent')[:5])
print(model.most_similar('terrible')[:5])

# 2. Classification avec des plongements lexicaux

On vous demande d’effectuer de la classification avec les plongements lexicaux obtenus.

## a) En reprenant le code développé dans le TP1 avec Scikitlearn, on vous demande cette fois de tester un modèle Naïve Bayes et de régression logistique avec des n-grammes (n=1,2,3 ensemble). Essayez de voir si une réduction de dimension améliore la classification. Ne fournissez que votre meilleur modèle. Evaluez vos algorithmes selon les métriques d’accuracy générale et de F1 par classe sur l’ensemble de test.

## b) En utilisant Tensorflow (ou Pytorch), on vous demande de développer un classificateur perceptron multicouches et un bi-LSTM avec les vecteurs d’un modèle word2vec pré-entrainé sur Wikipédia en Anglais (enwiki_upos_skipgram_300_3_2019) disponible à http://vectors.nlpl.eu/repository/11/3.zip. 

On s’attend à ce que vous effectuiez une moyenne des vecteurs de mots de chaque document pour obtenir un plongement du document.  

Evaluez vos algorithmes selon les métriques d’accuracy générale et de F1 par classe sur l’ensemble de test. Pour chacun des modèles, indiquez ses performances et ses spécifications (nombre d’époques, régularisation, optimiseur, nombre de couches, etc.). N’hésitez pas à expérimenter avec différents paramètres. Vous ne devez reporter que votre meilleure expérimentation.

## c) Ré-entrainez les modèles en b) avec vos propres vecteurs. Comparez maintenant la performance obtenue en en b) avec celles que vous obtenez en utilisant vos propres vecteurs de mots entrainés sur le corpus. 

## d) Générez une table ou un graphique qui regroupe les performances des modèles, leurs spécifications, la durée d’entraînement et commentez ces résultats. Quelle est l’influence des word embeddings sur les performances?  Quel est votre meilleur modèle ?